# SOCIO_DEMO: Party competition (synthetic) → EGT analysis

This notebook demonstrates the pipeline on **synthetic party competition**. We simulate seat/membership dynamics with a hidden replicator field, then attempt to recover a **strategy‑level interaction operator** `A` and **ESS**.

**Caveat:** This is illustrative. Real analyses must include intervention checks, nulls, and careful payoff definitions.


In [ ]:
# --- Canonical imports with fallbacks ---
try:
    import ts2eg as gm
except Exception:
    try:
        from gamify_timeseries import *  # standalone canonical file
        import types as _types
        gm = _types.SimpleNamespace(
            helmert_Q=helmert_Q, projectors=projectors,
            static_game_from_series=static_game_from_series,
            var_information_sharing_game=var_information_sharing_game,
            nmf_on_X=nmf_on_X, estimate_A_from_series=estimate_A_from_series,
            find_ESS=find_ESS
        )
    except Exception:
        raise ImportError("Neither 'ts2eg' package nor 'gamify_timeseries.py' (standalone) is available.")
try:
    from ts2eg import extensions as ext
except Exception:
    try:
        import egt_extensions as ext  # last resort
    except Exception:
        ext = None  # not all notebooks need extensions

import sys, os, numpy as np, pandas as pd, matplotlib.pyplot as plt
sys.path.append('/mnt/data')
from ts2eg import (
    growth_payoffs, info_gain_payoffs,
    estimate_A_from_series, find_ESS
)
print('Imports OK')

## 1) Simulate party competition
We simulate `N=4` parties and `T=240` monthly observations. A hidden skew‑dominant `A_true` induces rotation among archetypes; we convert mixture shares into **counts** with lognormal fluctuations.

In [ ]:
def simulate_parties(N=4, K=3, T=240, seed=0):
    rng = np.random.default_rng(seed)
    # True archetypes over parties
    S = rng.random((N, K)); S /= (np.linalg.norm(S, axis=0, keepdims=True) + 1e-12)
    # Skew-dominant interaction among archetypes (drives cycles)
    A_true = np.array([[0.0, 0.35, -0.25],
                       [-0.35, 0.0,  0.30],
                       [0.25, -0.30, 0.0]])
    x = np.full(K, 1.0/K)
    X_share = np.zeros((N, T))
    for t in range(T):
        X_share[:, t] = np.maximum(S @ x, 1e-8)
        X_share[:, t] /= X_share[:, t].sum()
        u = A_true @ x; u -= u.mean()
        dx = x * u
        x = x + 0.15*dx + 0.01*rng.random(K)
        x = np.maximum(x, 1e-8); x = x / x.sum()
    # Convert shares to counts with multiplicative noise
    base_pop = 1e6
    eps = np.exp(0.10*np.random.default_rng(1).standard_normal(T))
    total = base_pop * eps
    counts = (X_share * total)
    return S, X_share, counts

S_true, X_share, counts = simulate_parties()
parties = [f'Party_{i+1}' for i in range(X_share.shape[0])]
print('X_share:', X_share.shape, 'counts:', counts.shape)

## 2) Payoffs — per‑capita growth (canonical for group counts)

In [ ]:
v_growth = growth_payoffs(counts, dt=1.0, pad='edge')
print('v_growth shape:', v_growth.shape)
pd.DataFrame(v_growth, index=parties).iloc[:, :5]

## 3) Strategy basis via `nmf_on_X` on shares
We implement a compact multiplicative‑update NMF and normalize the columns of `S_hat`.

In [ ]:
# --- Canonical imports with fallbacks ---
try:
    import ts2eg as gm
except Exception:
    try:
        from gamify_timeseries import *  # standalone canonical file
        import types as _types
        gm = _types.SimpleNamespace(
            helmert_Q=helmert_Q, projectors=projectors,
            static_game_from_series=static_game_from_series,
            var_information_sharing_game=var_information_sharing_game,
            nmf_on_X=nmf_on_X, estimate_A_from_series=estimate_A_from_series,
            find_ESS=find_ESS
        )
    except Exception:
        raise ImportError("Neither 'ts2eg' package nor 'gamify_timeseries.py' (standalone) is available.")
try:
    from ts2eg import extensions as ext
except Exception:
    try:
        import egt_extensions as ext  # last resort
    except Exception:
        ext = None  # not all notebooks need extensions

from ts2eg import nmf_on_X
# Learn nonnegative strategy archetypes
K = 3
S, H = nmf_on_X(X, k=K, iters=300, seed=1, normalize='l2')
S = S  # columns already normalized by nmf_on_X
print('S shape:', S.shape)


## 4) Estimate `A` and search for ESS (growth payoffs)

In [ ]:
est = estimate_A_from_series(S, X_share, v_growth, k=K, lambda_=1e-2)
A = est['A']; R2 = est['R2']
print('R^2:', round(R2,3))
ess = [r for r in find_ESS(A, tol=1e-8, max_support=K) if r['is_ess']]
print('ESS count:', len(ess))
for r in ess:
    print('ESS support:', r['support'], 'x*=', np.round(r['x'], 3))
As, Aa = 0.5*(A + A.T), 0.5*(A - A.T)
print('||A_s||_F, ||A_a||_F =', float(np.linalg.norm(As)), float(np.linalg.norm(Aa)))

### Plot inferred strategy mixture `x(t)`

In [ ]:
plt.figure(figsize=(8,3))
for i in range(K):
    plt.plot(est['Xk'][i], label=f'x_{i+1}')
plt.title('Inferred strategy memberships'); plt.legend(); plt.show()

## 5) Alternative payoff: information‑gain to a target
We synthesize a target (e.g., *future nationwide approval/vote swing*) and compute rolling LOFO ablations.

In [ ]:
rng = np.random.default_rng(5)
w = np.array([0.6, -0.4, 0.2, -0.1])
J = (X_share.T @ w) + 0.05*rng.standard_normal(X_share.shape[1])
v_info = info_gain_payoffs(X_share, J, ridge=1e-2, window=48)
est_info = estimate_A_from_series(S, X_share, v_info, k=K, lambda_=1e-2)
print('R^2 (info):', round(est_info['R2'],3))
ess_info = [r for r in find_ESS(est_info['A'], tol=1e-8, max_support=K) if r['is_ess']]
print('ESS count (info):', len(ess_info))

## 6) Null test: circularly shift each row of `X_share`
This breaks cross‑group coordination while preserving row-wise autocorrelation.

In [ ]:
def circular_shift_rows(M):
    N, T = M.shape
    rng = np.random.default_rng(11)
    Y = np.zeros_like(M)
    for i in range(N):
        k = int(rng.integers(0, T))
        Y[i] = np.roll(M[i], k)
    return Y

X_null = circular_shift_rows(X_share)
S_null, _ = nmf_multiplicative(X_null, r=K, iters=300, seed=7)
S_null = S_null / (np.linalg.norm(S_null, axis=0, keepdims=True) + 1e-12)
est_null = estimate_A_from_series(S_null, X_null, v_growth, k=K, lambda_=1e-2)
print('R^2 (null):', round(est_null['R2'],3))
ess_null = [r for r in find_ESS(est_null['A'], tol=1e-8, max_support=K) if r['is_ess']]
print('ESS count (null):', len(ess_null))